# LangGraph Examples: Building Advanced AI Agent Workflows

This notebook provides hands-on examples of using LangGraph for building sophisticated AI agent workflows with complex control flow. You can run these examples to gain practical experience with this powerful framework.

## Helpful Resources

- [Official LangGraph Documentation](https://python.langchain.com/docs/langgraph)
- [LangGraph GitHub Repository](https://github.com/langchain-ai/langgraph)
- [LangGraph Concepts Guide](https://python.langchain.com/docs/langgraph/concepts)
- [LangGraph Tutorial](https://python.langchain.com/docs/langgraph/quick_start)
- [LangGraph API Reference](https://api.python.langchain.com/en/latest/langgraph_api_reference.html)
- [LangChain Blog: LangGraph](https://blog.langchain.dev/langgraph/)

## Setup Instructions

First, let's install the necessary packages:

In [ ]:
# Install required packages
!pip install langgraph langchain openai langchain_openai wikipedia

# Optional packages for visualization
!pip install graphviz

## Setting API Keys and Configurations

For these examples to work, you'll need to set your API keys for OpenAI or Azure OpenAI. Replace the placeholders with your actual API keys.

In [ ]:
import os

# Option 1: OpenAI Setup
# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual API key

# Option 2: Azure OpenAI Setup
# Set your Azure OpenAI variables
# os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-openai-api-key"  # Replace with your Azure API key
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com/"  # Replace with your endpoint

# Function to create LLM based on provider choice
def get_llm(provider="openai", temperature=0.0):
    if provider == "azure":
        from langchain_openai import AzureChatOpenAI
        return AzureChatOpenAI(
            temperature=temperature,
            azure_deployment="gpt-4",  # The deployment name you chose when you deployed the GPT-4 model
            openai_api_version="2023-05-15",
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
            api_key=os.environ.get("AZURE_OPENAI_API_KEY", "")
        )
    else:  # Default to OpenAI
        from langchain_openai import ChatOpenAI
        return ChatOpenAI(
            model="gpt-4",
            temperature=temperature
        )

# Choose your provider here
provider = "openai"  # Change to "azure" to use Azure OpenAI

# Get appropriate LLM
llm = get_llm(provider)

## Example 1: Basic Graph with Conditional Paths

Let's start with a simple graph that demonstrates conditional branching based on question difficulty.

In [ ]:
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from typing import Dict, TypedDict
import json

# Define our state using TypedDict for type hints
class QuestionState(TypedDict):
    question: str
    difficulty: str
    response: str

# Initialize the configurable LLM
llm = ChatOpenAI()

# Define nodes in the graph
def analyze_question(state: QuestionState) -> QuestionState:
    """Analyze the question and determine its difficulty."""
    response = llm.invoke([HumanMessage(content=f"Analyze this question and classify it as 'easy', 'medium', or 'hard': {state['question']}")])
    state["difficulty"] = response.content.strip().lower()
    return state

def answer_easy_question(state: QuestionState) -> QuestionState:
    """Process easy questions."""
    response = llm.invoke([HumanMessage(content=f"Please answer this easy question briefly: {state['question']}")])
    state["response"] = response.content
    return state

def answer_complex_question(state: QuestionState) -> QuestionState:
    """Process medium or hard questions with more detail."""
    response = llm.invoke([HumanMessage(content=f"Please answer this {state['difficulty']} question in detail, with examples: {state['question']}")])
    state["response"] = response.content
    return state

# Create the graph
workflow = StateGraph(QuestionState)

# Add nodes
workflow.add_node("analyze_question", analyze_question)
workflow.add_node("answer_easy", answer_easy_question)
workflow.add_node("answer_complex", answer_complex_question)

# Define conditional routing
def route_by_difficulty(state: QuestionState) -> str:
    if "easy" in state["difficulty"].lower():
        return "answer_easy"
    else:
        return "answer_complex"

# Add conditional edge
workflow.add_conditional_edges("analyze_question", route_by_difficulty)

# Set the entry point
workflow.set_entry_point("analyze_question")

# Compile the graph
graph = workflow.compile()

# Visualize the graph (if graphviz is installed)
try:
    graph.show()
except Exception as e:
    print(f"Could not visualize graph: {e}")

# Execute the graph with a sample question
result = graph.invoke({"question": "What is the capital of France?", "difficulty": "", "response": ""})

print(f"Question: {result['question']}")
print(f"Classified as: {result['difficulty']}")
print(f"\nResponse: {result['response']}")

## Example 2: Cyclic Graph for Iterative Process

Now let's create a more complex graph with cyclic behavior for iterative reasoning.

In [ ]:
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from typing import Dict, List, TypedDict

# Define our state using TypedDict
class SolverState(TypedDict):
    problem: str
    solution: str
    iterations: int
    max_iterations: int
    complete: bool

# Define nodes in the graph
def solve_step(state: SolverState) -> SolverState:
    """Generate or improve a solution."""
    llm = ChatOpenAI()
    
    # First iteration: create initial solution
    if state["iterations"] == 0:
        prompt = f"Generate an initial solution for this problem: {state['problem']}"
    # Subsequent iterations: refine the solution
    else:
        prompt = f"""
        Problem: {state['problem']}
        Current solution (iteration {state['iterations']}): {state['solution']}
        
        Improve the solution above by being more precise, addressing more edge cases, or fixing any issues.
        """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["solution"] = response.content
    return state

def evaluate_solution(state: SolverState) -> SolverState:
    """Evaluate if the solution is complete or needs improvement."""
    llm = ChatOpenAI()
    
    # Increment the iteration counter
    state["iterations"] += 1
    
    # If we've reached max iterations, mark as complete
    if state["iterations"] >= state["max_iterations"]:
        state["complete"] = True
        return state
        
    # Otherwise, evaluate the solution quality
    prompt = f"""
    Problem: {state['problem']}
    Current solution: {state['solution']}
    
    Evaluate this solution. Is it complete and optimal? 
    Answer only 'COMPLETE' if the solution is excellent and needs no further improvement.
    Answer 'INCOMPLETE' if the solution could be improved further.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["complete"] = "COMPLETE" in response.content.upper()
    return state

# Define condition for the loop
def should_continue(state: SolverState) -> str:
    """Determine whether to continue iterating or finish."""
    if state["complete"]:
        return "end"
    else:
        return "solve_step"

# Create the graph
workflow = StateGraph(SolverState)

# Add nodes
workflow.add_node("solve_step", solve_step)
workflow.add_node("evaluate", evaluate_solution)

# Add edges, including the cycle
workflow.add_edge("solve_step", "evaluate")
workflow.add_conditional_edges("evaluate", should_continue)
workflow.add_edge("end", None)  # End node

# Set the entry point
workflow.set_entry_point("solve_step")

# Visualize the cyclic graph
try:
    graph = workflow.compile()
    graph.show()
except Exception as e:
    print(f"Could not visualize graph: {e}")
    graph = workflow.compile()

# Execute the graph with a sample problem
result = graph.invoke({
    "problem": "Design an algorithm to find the maximum sum subarray in a given array of integers.",
    "solution": "",
    "iterations": 0,
    "max_iterations": 3,
    "complete": False
})

print(f"Problem: {result['problem']}")
print(f"Completed in {result['iterations']} iterations")
print(f"\nFinal Solution: {result['solution']}")

## Example 3: Decision-Making Agent with Research Capabilities

Let's create a more practical example of an agent that can research information and make decisions based on that research.

In [ ]:
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from typing import Dict, List, TypedDict, Optional
import textwrap

# Set up the Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Define our state using TypedDict
class ResearchState(TypedDict):
    query: str
    research_needed: bool
    research_results: Optional[str]
    follow_up_questions: List[str]
    final_answer: str

# Define nodes in the graph
def analyze_query(state: ResearchState) -> ResearchState:
    """Analyze the query to determine if research is needed."""
    llm = ChatOpenAI()
    
    prompt = f"""
    Query: {state['query']}
    
    Does this query require looking up factual information that might not be in your training data?
    Answer with just 'Yes' or 'No'.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["research_needed"] = response.content.strip().lower() == "yes"
    return state

def perform_research(state: ResearchState) -> ResearchState:
    """Use Wikipedia to research the query."""
    try:
        result = wikipedia.run(state["query"])
        # Truncate very long results
        if len(result) > 3000:
            result = result[:3000] + "... [truncated]"
        state["research_results"] = result
    except Exception as e:
        state["research_results"] = f"Error performing research: {str(e)}"
    return state

def generate_follow_up_questions(state: ResearchState) -> ResearchState:
    """Generate follow-up questions based on research."""
    if not state["research_needed"] or not state["research_results"]:
        state["follow_up_questions"] = []
        return state
    
    llm = ChatOpenAI()
    
    prompt = f"""
    Based on the following research results for the query: {state['query']}
    
    Research: {state['research_results']}
    
    Generate 3 specific follow-up questions that would help clarify or expand on this information.
    Return ONLY the questions as a numbered list.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    # Parse the questions (assuming they're numbered)
    lines = response.content.strip().split('\n')
    questions = [line.strip() for line in lines if line.strip()]
    state["follow_up_questions"] = questions
    return state

def formulate_answer(state: ResearchState) -> ResearchState:
    """Generate a comprehensive answer."""
    llm = ChatOpenAI()
    
    if state["research_needed"] and state["research_results"]:
        prompt = f"""
        Query: {state['query']}
        Research results: {state['research_results']}
        
        Based on this research, provide a comprehensive and accurate answer to the original query.
        """
    else:
        prompt = f"Please answer this query based on your knowledge: {state['query']}"
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["final_answer"] = response.content
    return state

# Define routing logic
def route_after_analysis(state: ResearchState) -> str:
    if state["research_needed"]:
        return "perform_research"
    else:
        return "formulate_answer"

# Create the graph
workflow = StateGraph(ResearchState)

# Add nodes
workflow.add_node("analyze_query", analyze_query)
workflow.add_node("perform_research", perform_research)
workflow.add_node("generate_follow_up_questions", generate_follow_up_questions)
workflow.add_node("formulate_answer", formulate_answer)

# Add edges
workflow.add_conditional_edges("analyze_query", route_after_analysis)
workflow.add_edge("perform_research", "generate_follow_up_questions")
workflow.add_edge("generate_follow_up_questions", "formulate_answer")

# Set the entry point
workflow.set_entry_point("analyze_query")

# Compile the graph
graph = workflow.compile()

# Try to visualize
try:
    graph.show()
except Exception as e:
    print(f"Could not visualize graph: {e}")

# Execute the graph with a query that likely needs research
result = graph.invoke({
    "query": "What were the key contributions of Marie Curie to science?", 
    "research_needed": False, 
    "research_results": None,
    "follow_up_questions": [],
    "final_answer": ""
})

print(f"Query: {result['query']}")
print(f"Research needed: {result['research_needed']}")
print("\nFollow-up questions:")
for i, question in enumerate(result['follow_up_questions'], 1):
    print(f"{i}. {question}")
print("\nFinal answer:")
print(textwrap.fill(result['final_answer'], width=100))

## Example 4: Multi-Agent System with State Persistence

Let's create a complex multi-agent system where different specialized agents collaborate with persistent state.

In [ ]:
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from typing import Dict, List, TypedDict, Optional, Any
import json

# Define our state using TypedDict
class ProjectState(TypedDict):
    task: str
    current_phase: str
    requirements: List[str]
    design: str
    implementation: str
    feedback: List[str]
    final_solution: str
    complete: bool
    
# Define agent roles and their corresponding functions

def product_manager(state: ProjectState) -> ProjectState:
    """Product manager agent that defines requirements."""
    llm = ChatOpenAI()
    
    prompt = f"""
    You are a Product Manager responsible for defining clear requirements.
    
    Task: {state['task']}
    
    Please define 3-5 specific, measurable requirements for this task.
    Format each requirement as a separate point.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    lines = response.content.strip().split('\n')
    requirements = [line.strip() for line in lines if line.strip()]
    state["requirements"] = requirements
    state["current_phase"] = "requirements_defined"
    return state

def designer(state: ProjectState) -> ProjectState:
    """Designer agent that creates a high-level design."""
    llm = ChatOpenAI()
    
    requirements_text = "\n".join([f"- {req}" for req in state["requirements"]])
    
    prompt = f"""
    You are a Designer responsible for creating a high-level design based on requirements.
    
    Task: {state['task']}
    
    Requirements:
    {requirements_text}
    
    Please create a high-level design that addresses these requirements.
    Include key components, their interactions, and any important design patterns or principles.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["design"] = response.content
    state["current_phase"] = "design_created"
    return state

def developer(state: ProjectState) -> ProjectState:
    """Developer agent that implements the design."""
    llm = ChatOpenAI()
    
    requirements_text = "\n".join([f"- {req}" for req in state["requirements"]])
    
    prompt = f"""
    You are a Developer responsible for implementing the design.
    
    Task: {state['task']}
    
    Requirements:
    {requirements_text}
    
    Design:
    {state['design']}
    
    Please provide an implementation (code or pseudocode) that follows this design 
    and meets all the requirements. Focus on clarity and correctness.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["implementation"] = response.content
    state["current_phase"] = "implementation_complete"
    return state

def reviewer(state: ProjectState) -> ProjectState:
    """Reviewer agent that evaluates the implementation."""
    llm = ChatOpenAI()
    
    requirements_text = "\n".join([f"- {req}" for req in state["requirements"]])
    
    prompt = f"""
    You are a Reviewer responsible for evaluating the implementation against requirements.
    
    Task: {state['task']}
    
    Requirements:
    {requirements_text}
    
    Implementation:
    {state['implementation']}
    
    Please review the implementation and provide 2-3 specific pieces of feedback.
    Focus on whether it meets all requirements and follows the design.
    Format each feedback point separately.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    lines = response.content.strip().split('\n')
    feedback = [line.strip() for line in lines if line.strip()]
    state["feedback"] = feedback
    state["current_phase"] = "review_complete"
    return state

def integrator(state: ProjectState) -> ProjectState:
    """Integrator agent that finalizes the solution based on feedback."""
    llm = ChatOpenAI()
    
    feedback_text = "\n".join([f"- {fb}" for fb in state["feedback"]])
    
    prompt = f"""
    You are an Integrator responsible for finalizing the solution based on all previous work.
    
    Task: {state['task']}
    Implementation: {state['implementation']}
    Feedback: 
    {feedback_text}
    
    Please create a final solution that incorporates the feedback and ensures all requirements are met.
    This should be the complete, polished solution ready for delivery.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    state["final_solution"] = response.content
    state["complete"] = True
    state["current_phase"] = "complete"
    return state

# Define workflow transitions based on current phase
def route_by_phase(state: ProjectState) -> str:
    current_phase = state["current_phase"]
    
    phase_routing = {
        "start": "product_manager",
        "requirements_defined": "designer",
        "design_created": "developer",
        "implementation_complete": "reviewer",
        "review_complete": "integrator",
        "complete": "end"
    }
    
    return phase_routing.get(current_phase, "product_manager")

# Create the graph
workflow = StateGraph(ProjectState)

# Add nodes for each agent role
workflow.add_node("product_manager", product_manager)
workflow.add_node("designer", designer)
workflow.add_node("developer", developer)
workflow.add_node("reviewer", reviewer)
workflow.add_node("integrator", integrator)

# Add conditional routing based on the current phase
workflow.add_conditional_edges("start", route_by_phase)
workflow.add_conditional_edges("product_manager", route_by_phase)
workflow.add_conditional_edges("designer", route_by_phase)
workflow.add_conditional_edges("developer", route_by_phase)
workflow.add_conditional_edges("reviewer", route_by_phase)
workflow.add_conditional_edges("integrator", route_by_phase)
workflow.add_edge("end", None)

# Set the entry point
workflow.set_entry_point("start")

# Compile the graph
graph = workflow.compile()

# Try to visualize
try:
    graph.show()
except Exception as e:
    print(f"Could not visualize graph: {e}")

# Execute the graph with a sample task
result = graph.invoke({
    "task": "Create a simple todo list application with add, remove, and mark-complete functionality",
    "current_phase": "start",
    "requirements": [],
    "design": "",
    "implementation": "",
    "feedback": [],
    "final_solution": "",
    "complete": False
})

print(f"Task: {result['task']}")
print("\nRequirements:")
for req in result['requirements']:
    print(f"- {req}")
    
print("\nDesign:")
print(result['design'][:300] + "..." if len(result['design']) > 300 else result['design'])

print("\nImplementation (excerpt):")
print(result['implementation'][:300] + "..." if len(result['implementation']) > 300 else result['implementation'])

print("\nFeedback:")
for fb in result['feedback']:
    print(f"- {fb}")
    
print("\nFinal Solution (excerpt):")
print(result['final_solution'][:300] + "..." if len(result['final_solution']) > 300 else result['final_solution'])

## Further Exploration

These examples demonstrate the core capabilities of LangGraph for creating complex, stateful workflows. For more advanced usage, consider exploring:

1. **Persistent State Management**: Using databases or file systems to maintain state between sessions
2. **Advanced Visualization**: Creating custom visualizations of complex workflows
3. **Integration with External Systems**: Connecting nodes to APIs, databases, etc.
4. **Nested Graphs**: Creating hierarchical workflows with graphs inside nodes
5. **Error Handling**: Implementing robust error recovery mechanisms

Check out the [LangGraph documentation](https://python.langchain.com/docs/langgraph) for more examples and advanced usage scenarios.

## Using Azure OpenAI with LangGraph

This example demonstrates how to explicitly create and use Azure OpenAI with LangGraph for a cyclic reasoning pattern:

In [ ]:
# Azure OpenAI specific setup
from langchain_openai import AzureChatOpenAI

# Create Azure OpenAI LLM instance (comment out if you don't have Azure OpenAI access)
azure_llm = AzureChatOpenAI(
    temperature=0.7,
    azure_deployment="gpt-4",  # Your deployment name
    openai_api_version="2023-05-15",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://your-resource-name.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key")
)

# Define our state
class IterativeState(TypedDict):
    problem: str
    solution: str
    iterations: int
    max_iterations: int
    complete: bool

# Define nodes for our reasoning graph
def refine_solution(state: IterativeState):
    """Refine the current solution."""
    # Using Azure OpenAI for this function
    prompt = f"""
    Problem: {state['problem']}
    Current solution: {state['solution']}
    
    You are in iteration {state['iterations']} of solving this problem.
    Please refine and improve the solution, making it more detailed and accurate.
    """
    
    # Uncomment the next two lines to use Azure OpenAI when properly configured
    # response = azure_llm.invoke([HumanMessage(content=prompt)])
    # state["solution"] = response.content
    
    # For demonstration, use the regular LLM instead
    response = llm.invoke([HumanMessage(content=prompt)])
    state["solution"] = response.content
    
    return state

def evaluate_solution(state: IterativeState):
    """Evaluate if the solution is complete."""
    state["iterations"] += 1
    
    if state["iterations"] >= state["max_iterations"]:
        state["complete"] = True
        return state
    
    prompt = f"""
    Problem: {state['problem']}
    Current solution: {state['solution']}
    
    Is this solution complete and optimal? Answer YES if it is complete, or NO with a brief explanation if it needs further refinement.
    """
    
    # Uncomment the next line to use Azure OpenAI when properly configured
    # response = azure_llm.invoke([HumanMessage(content=prompt)])
    
    # For demonstration, use the regular LLM instead
    response = llm.invoke([HumanMessage(content=prompt)])
    
    if "YES" in response.content.upper() or state["iterations"] >= state["max_iterations"]:
        state["complete"] = True
    else:
        state["complete"] = False
        
    return state

# Create the graph
workflow = StateGraph(IterativeState)

# Add nodes
workflow.add_node("refine", refine_solution)
workflow.add_node("evaluate", evaluate_solution)

# Add edges with conditional logic
workflow.add_edge("refine", "evaluate")
workflow.add_edge("evaluate", "refine", condition=lambda state: not state["complete"])

# Set the entry point
workflow.set_entry_point("refine")

# Compile the graph
iterative_graph = workflow.compile()

# For demonstration purposes (uncomment to execute)
'''
# Create initial state
initial_state = {
    "problem": "Design an efficient algorithm to find the longest palindromic substring in a string.",
    "solution": "We can use a simple approach by checking all possible substrings.",
    "iterations": 0,
    "max_iterations": 3,
    "complete": False
}

# Execute the graph
result = iterative_graph.invoke(initial_state)

print(f"Final solution after {result['iterations']} iterations:")
print(result['solution'])
'''

## Complete Example with Configurable LLM Choice

This example brings everything together in a complete implementation that can easily switch between OpenAI and Azure OpenAI:

In [ ]:
# Complete Implementation with Provider Choice

def run_research_agent(query, provider="openai", max_iterations=3):
    """Run a research agent workflow with the specified LLM provider."""
    # Get the right LLM
    if provider == "azure":
        from langchain_openai import AzureChatOpenAI
        llm = AzureChatOpenAI(
            temperature=0.7,
            azure_deployment="gpt-4",
            openai_api_version="2023-05-15",
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
            api_key=os.environ.get("AZURE_OPENAI_API_KEY", "")
        )
    else:  # Default to OpenAI
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(
            model="gpt-4",
            temperature=0.7
        )
    
    # Define our state
    class ResearchState(TypedDict):
        query: str
        plan: str
        research_notes: str
        final_report: str
        iterations: int
        max_iterations: int
        complete: bool
    
    # Define nodes
    def create_plan(state: ResearchState):
        """Create a research plan based on the query."""
        prompt = f"Create a detailed research plan for investigating: {state['query']}"
        response = llm.invoke([HumanMessage(content=prompt)])
        state["plan"] = response.content
        return state
    
    def conduct_research(state: ResearchState):
        """Conduct research based on the plan."""
        prompt = f"""
        Research Query: {state['query']}
        Research Plan: {state['plan']}
        Current Research Notes: {state.get('research_notes', '')}
        Iteration: {state['iterations'] + 1} of {state['max_iterations']}
        
        Based on the above, conduct the next phase of research. Add new information and insights not already covered.
        """
        response = llm.invoke([HumanMessage(content=prompt)])
        
        # Append new research to existing notes
        if 'research_notes' not in state or not state['research_notes']:
            state['research_notes'] = response.content
        else:
            state['research_notes'] = state['research_notes'] + "\n\n" + response.content
            
        state['iterations'] += 1
        return state
    
    def evaluate_research(state: ResearchState):
        """Evaluate if research is complete."""
        if state['iterations'] >= state['max_iterations']:
            state['complete'] = True
            return state
            
        prompt = f"""
        Research Query: {state['query']}
        Research Plan: {state['plan']}
        Current Research Notes: {state['research_notes']}
        
        Is the research complete enough to write a comprehensive report? Answer YES or NO.
        If NO, briefly explain what aspects need more investigation.
        """
        
        response = llm.invoke([HumanMessage(content=prompt)])
        
        if "YES" in response.content.upper():
            state['complete'] = True
        else:
            state['complete'] = False
            
        return state
    
    def generate_report(state: ResearchState):
        """Generate final research report."""
        prompt = f"""
        Research Query: {state['query']}
        Research Plan: {state['plan']}
        Research Notes: {state['research_notes']}
        
        Create a comprehensive, well-structured final report based on the research conducted.
        The report should be informative, factual, and directly address the original query.
        """
        
        response = llm.invoke([HumanMessage(content=prompt)])
        state['final_report'] = response.content
        return state
    
    # Create the graph
    workflow = StateGraph(ResearchState)
    
    # Add nodes
    workflow.add_node("create_plan", create_plan)
    workflow.add_node("conduct_research", conduct_research)
    workflow.add_node("evaluate_research", evaluate_research)
    workflow.add_node("generate_report", generate_report)
    
    # Add edges with conditional logic
    workflow.add_edge("create_plan", "conduct_research")
    workflow.add_edge("conduct_research", "evaluate_research")
    workflow.add_edge("evaluate_research", "conduct_research", condition=lambda state: not state["complete"])
    workflow.add_edge("evaluate_research", "generate_report", condition=lambda state: state["complete"])
    
    # Set the entry point
    workflow.set_entry_point("create_plan")
    
    # Compile the graph
    research_graph = workflow.compile()
    
    # Create initial state
    initial_state = {
        "query": query,
        "plan": "",
        "research_notes": "",
        "final_report": "",
        "iterations": 0,
        "max_iterations": max_iterations,
        "complete": False
    }
    
    # Run the graph
    return research_graph.invoke(initial_state)

# Example usage (uncomment to run)
'''
query = "What are the most promising applications of graph neural networks in drug discovery?"
provider = "openai"  # Change to "azure" to use Azure OpenAI
result = run_research_agent(query, provider=provider, max_iterations=2)

print("========== FINAL RESEARCH REPORT ==========\n")
print(result["final_report"])
'''

## Conclusion

In this notebook, we've explored LangGraph's powerful features for building complex AI agent workflows with advanced control flow. We've demonstrated:

1. Setting up LangGraph with both OpenAI and Azure OpenAI
2. Creating conditional branching in workflows based on LLM decisions
3. Implementing cyclic reasoning patterns with state management
4. Building a complete research agent with configurable LLM backend

The examples show how LangGraph enables sophisticated graph-based architectures that can represent complex, non-linear workflows with advanced state management capabilities.